In [1]:
import optuna
import os
import trako as TKO
import numpy as np

In [2]:
import imp
tkompare = imp.load_source('tkompare', '../tkompare')
untrakofy = imp.load_source('untrakofy', '../untrakofy')

In [3]:
DATASETNAME = 'ADHD_STFC_diff_for_Daniel'
DATADIR = '/home/d/DATA/Dropbox/TRAKODATA/'+DATASETNAME+'/'
SUBJECTS = ['101','201']
GROUP = ['tracts_left_hemisphere']

In [4]:
def trakofy_test(trial):
    
    vtpfile = trial.study.user_attrs['vtpfile']
    tkofile = trial.study.user_attrs['tkofile']
    
    position = trial.suggest_categorical('position', [True, False])
    sequential = trial.suggest_categorical('sequential', [True, False])
    quantization_bits = trial.suggest_int('quantization_bits', 0, 31)
    compression_level = trial.suggest_int('compression_level', 0, 10)
#     quantization_bits = trial.suggest_discrete_uniform('quantization_bits', 0, 31, 1)
#     compression_level = trial.suggest_discrete_uniform('compression_level', 0, 10, 1)
#     quantization_range = 
#     quantization_origin = 
    
    config = {
      '*': { 
        'position':position,
        'sequential':sequential,
        'quantization_bits':quantization_bits,
        'compression_level':compression_level,
        'quantization_range':-1,
        'quantization_origin':None
      }

    }
    
#     print(config['position'], config['sequential'], config['quantization_bits'], config['compression_level'])
    
    try:
        tkoed = TKO.Encoder.fromVtp(vtpfile, config=config, verbose=False)
        tkoed.save(tkofile)
    except:
        raise optuna.exceptions.TrialPruned()

    tkosize = os.path.getsize(tkofile)

    try:
        restoredfile = '/tmp/restored.vtp'
        untrakofy.untrakofy(tkofile, restoredfile, verbose=False)
    except:
        raise optuna.exceptions.TrialPruned()
    
    
    return tkosize
    
    

In [ ]:
OUTPUTDIR = '/tmp/TRAKO_OPTUNA1/'
vtpsizes = []
tkosizes = []
for S in SUBJECTS[0:1]:
    for G in GROUP[0:1]:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)[0:1]
        
        if not os.path.exists(CURRENTOUTDIR):
            os.makedirs(CURRENTOUTDIR)
        
        for f in files:
            if not f.endswith('vtp'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)
            vtpsize = os.path.getsize(vtpfile)
            tkofile = os.path.join(CURRENTOUTDIR, f.replace('vtp', 'tko'))
            
            print('VTP size', vtpsize)
            
            tkoed = TKO.Encoder.fromVtp(vtpfile, config=None, verbose=False)
            tkoed.save(tkofile)
            tkosize = os.path.getsize(tkofile)
            
            print('TKO default size', tkosize)
            
            study = optuna.create_study()
            study.set_user_attr('vtpfile', vtpfile)
            study.set_user_attr('tkofile', tkofile)
            study.optimize(trakofy_test, n_trials=100)
            

VTP size 1172155
TKO default size 377009


[I 2020-01-16 10:40:44,309] Setting status of trial#0 as TrialState.PRUNED. 
[I 2020-01-16 10:40:53,130] Setting status of trial#1 as TrialState.PRUNED. 
[I 2020-01-16 10:40:53,395] Setting status of trial#2 as TrialState.PRUNED. 
[I 2020-01-16 10:40:53,642] Setting status of trial#3 as TrialState.PRUNED. 
[I 2020-01-16 10:40:53,931] Setting status of trial#4 as TrialState.PRUNED. 
[I 2020-01-16 10:40:54,521] Setting status of trial#5 as TrialState.PRUNED. 
[I 2020-01-16 10:40:54,767] Setting status of trial#6 as TrialState.PRUNED. 
[I 2020-01-16 10:40:55,029] Setting status of trial#7 as TrialState.PRUNED. 
[I 2020-01-16 10:40:55,462] Setting status of trial#8 as TrialState.PRUNED. 
[I 2020-01-16 10:40:55,814] Setting status of trial#9 as TrialState.PRUNED. 
[I 2020-01-16 10:40:56,067] Setting status of trial#10 as TrialState.PRUNED. 
[I 2020-01-16 10:41:24,066] Setting status of trial#11 as TrialState.PRUNED. 
[I 2020-01-16 10:41:24,320] Setting status of trial#12 as TrialState.PRUNE

In [ ]:
study.best_trial

In [ ]:
study.best_trial.params

In [ ]:
config = {
  '*': { 
    'position':study.best_trial.params['position'],
    'sequential':study.best_trial.params['sequential'],
    'quantization_bits':study.best_trial.params['quantization_bits'],
    'compression_level':study.best_trial.params['compression_level'],
    'quantization_range':-1,
    'quantization_origin':None
  }

}



# tkompare.compare(vtpfile, restoredfile)

In [ ]:
tkofile

In [ ]:
vtpfile